In [43]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 

In [30]:
cd '/Users/admin/Jedha/'

'/Users/admin/Jedha'

In [38]:
from s3_credentials import *

In [41]:
import boto3
session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")

#for bucket in s3.buckets.all():
#        print(bucket.name)

In [53]:
# real
obj = s3.Object('jedha-fake-reviews-project', "datasets/real_reviews_raw.csv")
real = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# fake
obj = s3.Object('jedha-fake-reviews-project', "datasets/fake_reviews_raw.csv")
fake = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)




In [56]:
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

pivot_table = real.loc[:, ['url','restaurant_average_rating','restaurant_reviews_count','restaurant_expensiveness','restaurant_name']].drop_duplicates('url')

fake_merged = (pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left'))
fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered]

dataset = pd.concat([real, fake_dataset_reworked])
dataset = full_dataset.reset_index(drop = True)
dataset = full_dataset.drop_duplicates()
dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))

In [57]:
dataset['text_length'] = dataset['text_review'].apply(lambda x : len(x))

In [58]:
def get_lang(x):

    try:
        lang = detect(str(x))
    except:
        lang = 'not_found'

    return lang

In [59]:
lang = []
for i in range(len(dataset)):
    item = dataset['text_review'].iloc[i]
    l = get_lang(item)
    lang.append(l)
    if i % 1500 == 0:
        print(i, 'ok')

KeyError: 'review_lite'

In [ ]:
dataset['lang'] = lang

In [31]:
dataset['id'] = dataset['username'] + str(dataset['user_friends_count']) + str(dataset['user_location']) + str(dataset['user_reviews_count']) + str(dataset['user_total_image_posted'])

In [19]:
print('real_review | mean_user_review_count: ', dataset.loc[dataset['is_real_review']==1]['user_reviews_count'].mean())
print('fake_review | mean_user_review_count: ', dataset.loc[dataset['is_real_review']==0]['user_reviews_count'].mean())
print('-----------------------------------------------------------------------------------------')
print('real_review | mean_user_friend_count: ', dataset.loc[dataset['is_real_review']==1]['user_friends_count'].mean())
print('fake_review | mean_user_friend__count: ', dataset.loc[dataset['is_real_review']==0]['user_friends_count'].mean())
print('-----------------------------------------------------------------------------------------')
print('real_review | mean_user_image_posted_count: ', dataset.loc[dataset['is_real_review']==1]['user_total_image_posted'].mean())
print('fake_review | mean_user_image_posted_count: ', dataset.loc[dataset['is_real_review']==0]['user_total_image_posted'].mean())


real_review | mean_user_review_count:  95.22160705003445
fake_review | mean_user_review_count:  19.7975533181769
-----------------------------------------------------------------------------------------
real_review | mean_user_friend_count:  43.63132381562564
fake_review | mean_user_friend__count:  240.25062249648155
-----------------------------------------------------------------------------------------
real_review | mean_user_image_posted_count:  161.56677103103127
fake_review | mean_user_image_posted_count:  58.29793340987371
